In [2]:
val slickVersion = "3.3.3"
val slickPgVersion = "0.19.5"
val postgresVersion = "42.2.19"
val logbackVersion = "1.2.3"

interp.load.ivy("com.typesafe.slick" %% "slick" % slickVersion)
interp.load.ivy("com.typesafe.slick" %% "slick-hikaricp" % slickVersion)
interp.load.ivy("com.github.tminglei" %% "slick-pg" % slickPgVersion)
interp.load.ivy("com.github.tminglei" %% "slick-pg_spray-json" % slickPgVersion)
interp.load.ivy("org.postgresql" % "postgresql" % postgresVersion)
interp.load.ivy("ch.qos.logback" % "logback-classic" % logbackVersion)

slickVersion: String = "3.3.3"
slickPgVersion: String = "0.19.5"
postgresVersion: String = "42.2.19"
logbackVersion: String = "1.2.3"

In [3]:
interp.load.cp(os.Path("/home/jovyan/jar/masterthesis-khipu_2.13-0.1.jar"))

In [5]:
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.Future
import scala.concurrent.duration._
import scala.concurrent.{ Await, Future }
import scala.language.postfixOps

import de.tmrdlt.database.workflowlist.WorkflowListDB
import de.tmrdlt.database.action.ActionDB


val workflowListDB = new WorkflowListDB()
val actionDB = new ActionDB()


val workflowLists =  Await.result(workflowListDB.getWorkflowLists, 2 seconds)
val actions =  Await.result(actionDB.getActions, 2 seconds)
val moveActions =  Await.result(actionDB.getMoveActions, 2 seconds)

import scala.concurrent.ExecutionContext.Implicits.global

import scala.concurrent.Future

import scala.concurrent.duration._

import scala.concurrent.{ Await, Future }

import scala.language.postfixOps


import de.tmrdlt.database.workflowlist.WorkflowListDB

import de.tmrdlt.database.action.ActionDB



workflowListDB: WorkflowListDB = de.tmrdlt.database.workflowlist.WorkflowListDB@96d9f87
actionDB: ActionDB = de.tmrdlt.database.action.ActionDB@5b31ec84
workflowLists: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
    233L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-02-26T09:50:01.786
  ),
  WorkflowList(
    234L,
    "5fad0555e165558a64ad0479",
    "Tasks / ToDos",
    None,
    Some(233L),
    0L,
    LIST,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-02-26T09:49:56.963
  ),
  WorkflowList(
    235L,
    "